# Project 2

# Used Vehicle Price Prediction

## Introduction

- 1.2 Million listings scraped from TrueCar.com - Price, Mileage, Make, Model dataset from Kaggle: [data](https://www.kaggle.com/jpayne/852k-used-car-listings)
- Each observation represents the price of an used car

In [1]:
%matplotlib inline
import pandas as pd

In [2]:
data = pd.read_csv('dataTrain_carListings.zip')

In [3]:
data.head()

,Price,Year,Mileage,State,Make,Model
0,21490,2014,31909,MD,Nissan,MuranoAWD
1,21250,2016,25741,KY,Chevrolet,CamaroCoupe
2,20925,2016,24633,SC,Hyundai,Santa
3,14500,2012,84026,OK,Jeep,Grand
4,32488,2013,22816,TN,Jeep,Wrangler


In [4]:
import warnings
warnings.filterwarnings("ignore")

# Exercise P2.1 (50%)

Develop a machine learning model that predicts the price of the of car using as an input ['Year', 'Mileage', 'State', 'Make', 'Model']

#### Evaluation:
- 25% - Performance of the models using a manually implemented K-Fold (K=10) cross-validation
- 25% - Notebook explaining the process for selecting the best model. You must specify how the calibration of each of the parameters is done and how these change the performance of the model. It is expected that a clear comparison will be made of all implemented models.. Present the most relevant conslusions about the whole process. 


#### Cargamos las variables y creamos dummies para las variables categóricas

In [5]:
X = data[['Year','Mileage','State','Make','Model']]
y = data['Price']

In [6]:
X['State'] = X['State'].str.strip()

In [7]:
X = pd.get_dummies(X)

#### Guardamos en un pickle los nombres de las columnas para usarlos posteriormente en la API

In [8]:
from sklearn.externals import joblib
joblib.dump(X.columns, 'columns_mod.pkl', compress=3)

['columns_mod.pkl']

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### XGBRegressor
#### Entrenamos un XGBRegressor y evaluamos su poder predictivo mediante el RMSE

In [10]:
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import datetime
from sklearn.model_selection import GridSearchCV

#### Los parametros usados a continuación fueron los mejores parámetros encontrados en ejecuciones anteriores, pero por cuestiones de practicidad se estima el modelo más reciente uncamente con los parámetros encontrados anteriormente.

In [11]:
xgb1 = XGBRegressor()

parameters = {'objective':['reg:squarederror'],
              'learning_rate': [0.07],
              'max_depth': [7],
              'subsample': [0.8],
              'colsample_bytree': [0.8],
              'n_estimators': [200]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 10,
                        n_jobs = -1,
                        verbose=True)

xgb_grid.fit(X_train,y_train)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed: 119.7min remaining: 79.8min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 127.4min finished


GridSearchCV(cv=10, error_score=nan,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estim...
                                    reg_lambda=None, scale_pos_weight=None,
                                    subsample=None, tree_method=None,
                                    validate_parameters=None, verbosity=None),
             iid='deprecated', n_jobs=-1,
             param_grid={

In [12]:
y_pred = xgb_grid.predict(X_test)

In [13]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred))
rmse

4960.446701461676

#### Guardamos el modelo entrenado en un archivo .pkl

In [14]:
joblib.dump(xgb_grid, 'price_xgb.pkl', compress=3)

['price_xgb.pkl']

#### Podemos determinar que el XGBRegressor entrenado tiene un RMSE de 4960.